In [ ]:
pip install tldextract

In [1]:
import json
from urllib.parse import urlparse
import pandas as pd
import tldextract

In [2]:
# Define the normalization function
def normalize_url(url):
    if url:
        # Remove 'http://', 'https://', and 'www.'
        url = url.lower().replace('http://', '').replace('https://', '').replace('www.', '')
        # Extract domain and suffix (e.g., 'example.com')
        domain = url.split('/')[0]  # Keep only the domain part
        return domain
    return None

In [3]:
test_url = 'https://www.fesbc.ca/about-the-fesbc/'

normalized_url = normalize_url(test_url)
normalized_url

'fesbc.ca'

In [4]:
test_url_2 = 'https://www.fesbc.ca/en/'

normalized_url = normalize_url(test_url_2)
normalized_url

'fesbc.ca'

In [5]:
import gzip
import json
from pathlib import Path

import httpx

http_client = httpx.AsyncClient(
    timeout=httpx.Timeout(read=60, write=60, connect=60, pool=60),
)

In [6]:
wip_dir = Path.home() / "Downloads"
pth_in_dump = wip_dir / "free_company_dataset.json.zip"

In [7]:
def get_df_in_raw(pth_in_dump: Path) -> pd.DataFrame:
    """Get the raw dataframe from the dump file"""
    cols = ["id", "website", "name", "founded", "size", "locality", "region", "country", "industry", "linkedin_url"]
    lst = []
    with gzip.open(pth_in_dump, "rt") as fp:
        for line in fp:
            dic = json.loads(line)
            lst.append(tuple(dic[key] for key in cols))
    return pd.DataFrame(lst, columns=cols)


async def download_to_file(url: str, pth: Path, http_client: httpx.AsyncClient):
    with pth.open("wb") as fp:
        async with http_client.stream("GET", url) as resp:
            async for chunk in resp.aiter_bytes():
                fp.write(chunk)

In [8]:
df_in_raw = get_df_in_raw(pth_in_dump)

In [16]:
# Read Atium CSV file into DataFrame
csv_file_path = wip_dir / 'companies.csv'
atium_df = pd.read_csv(csv_file_path)

In [17]:
len(atium_df)

27174

In [19]:
# Step 1: Exclude rows in atium_df with URLs containing sub-sections (except /en/)
# Allow URLs with trailing / but no additional sub-sections
atium_df = atium_df[~atium_df['url'].str.contains(r'\/[^\/]+\/', regex=True) | 
                  atium_df['url'].str.contains(r'\/en\/', regex=True) |
                  atium_df['url'].str.endswith('/')]

In [20]:
len(atium_df)

23529

In [21]:
atium_df.head(20)

,id,name,url
0,17794c4a-0208-4817-b2fe-819297c95d3e,Sound Ventures,https://soundventures.com
1,62b4b5b9-af62-4e4b-a192-87aedf306476,Instituto Kabu,https://www.kabu.org.br/
2,8117cfd3-d47f-471f-b27c-6431d0e37cf2,Taihe Institute,http://www.taiheglobal.org/en/
3,15b3d733-5f9f-4832-93f7-122ca510db83,Oceaneye.ch,http://www.oceaneye.ch
4,0f3d9a0e-d943-4d0f-844c-2104a62382ac,Centro de Estudos Rioterra,https://rioterra.org/
5,3c311db2-04b7-4701-ba5e-86c83d68d82a,Tortoise Media,https://www.tortoisemedia.com/
6,b4ec1771-2f86-4ebb-a2e5-45839b82b599,Quintas Renewable Energy Solutions,http://www.quintasenergies.com.ng/
7,3f7115e8-aa41-4f1e-a737-70054a515b4c,G7 litalia,https://www.g7italy.it/en/
9,109ab236-d853-4dc6-b7da-f03d8cde058c,Life Faerie Glen Hospital,https://www.lifehealthcare.co.za/hospitals/gau...
10,50909182-769c-48b1-9d7f-c5888156164d,Privy Council Office,https://www.royaloffice.th/about-royaloffice/%...


In [22]:
# Step 2: Normalize URLs in atium_df
atium_df['normalized_website'] = atium_df['url'].apply(normalize_url)


/var/folders/zr/fksnltgn6b5f322_h382g5yh0000gp/T/ipykernel_79965/4156729947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atium_df['normalized_website'] = atium_df['url'].apply(normalize_url)


In [23]:
atium_df.head()

,id,name,url,normalized_website
0,17794c4a-0208-4817-b2fe-819297c95d3e,Sound Ventures,https://soundventures.com,soundventures.com
1,62b4b5b9-af62-4e4b-a192-87aedf306476,Instituto Kabu,https://www.kabu.org.br/,kabu.org.br
2,8117cfd3-d47f-471f-b27c-6431d0e37cf2,Taihe Institute,http://www.taiheglobal.org/en/,taiheglobal.org
3,15b3d733-5f9f-4832-93f7-122ca510db83,Oceaneye.ch,http://www.oceaneye.ch,oceaneye.ch
4,0f3d9a0e-d943-4d0f-844c-2104a62382ac,Centro de Estudos Rioterra,https://rioterra.org/,rioterra.org


In [24]:
df_in_raw.head()

,id,website,name,founded,size,locality,region,country,industry,linkedin_url
0,CuyWJQh9OJallb2JWzbvswRxyU6h,guincho-almeida-lp.negocio.site,guincho almeida lp,NaN,1-10,None,None,brazil,individual & family services,linkedin.com/company/guincho-almeida-lp
1,DSvFD9Lb6FnKsgJXf2DIYQ1IwDNx,csiavocats.com,cabinet d'avocats saïd ibrahim,2000.0,1-10,None,None,comoros,law practice,linkedin.com/company/csiavocats
2,pBAJmryGZ0tcDsG13jT8RQi1GzFM,keychangenow.com,kci (key change institute),2017.0,1-10,san diego,california,united states,management consulting,linkedin.com/company/key-change-insitute
3,WmmByBqKxmgDSaWLXp0vJgJljsya,swimmingmoscow.com,swimming moscow,2017.0,1-10,moskva,novosibirsk,russia,sports,linkedin.com/company/swimming-moscow
4,Uf09EMyMNrjKFo4lV1bUSQSYnNY3,herbzy.space,herbzy,2020.0,1-10,None,None,poland,food & beverages,linkedin.com/company/herbzy


In [26]:
# Step 3: Filter df_in_raw to include only relevant rows
relevant_websites = set(atium_df['normalized_website'])
df_in_raw_filtered = df_in_raw[df_in_raw['website'].isin(relevant_websites)]

In [27]:
# Step 4: Create a dictionary mapping normalized_website to linkedin_url, region, country, and industry
website_to_data = df_in_raw_filtered.set_index('website')[['linkedin_url', 'region', 'country', 'industry']].to_dict(orient='index')

In [28]:
# Step 5: Enrich atiu_df with linkedin_url, region, country, and industry
atium_df['linkedin_url'] = atium_df['normalized_website'].map(lambda x: website_to_data.get(x, {}).get('linkedin_url'))
atium_df['region'] = atium_df['normalized_website'].map(lambda x: website_to_data.get(x, {}).get('region'))
atium_df['country'] = atium_df['normalized_website'].map(lambda x: website_to_data.get(x, {}).get('country'))
atium_df['industry'] = atium_df['normalized_website'].map(lambda x: website_to_data.get(x, {}).get('industry'))

In [29]:
atium_df.head()

,id,name,url,normalized_website,linkedin_url,region,country,industry
0,17794c4a-0208-4817-b2fe-819297c95d3e,Sound Ventures,https://soundventures.com,soundventures.com,linkedin.com/company/soundwavesholdings,california,united states,venture capital & private equity
1,62b4b5b9-af62-4e4b-a192-87aedf306476,Instituto Kabu,https://www.kabu.org.br/,kabu.org.br,linkedin.com/company/instituto-kabu,para,brazil,non-profit organization management
2,8117cfd3-d47f-471f-b27c-6431d0e37cf2,Taihe Institute,http://www.taiheglobal.org/en/,taiheglobal.org,linkedin.com/company/taiheglobalinstitute,beijing,china,think tanks
3,15b3d733-5f9f-4832-93f7-122ca510db83,Oceaneye.ch,http://www.oceaneye.ch,oceaneye.ch,linkedin.com/company/oceaneye-ch,geneva,switzerland,research
4,0f3d9a0e-d943-4d0f-844c-2104a62382ac,Centro de Estudos Rioterra,https://rioterra.org/,rioterra.org,None,None,None,None


In [30]:
# Step 6: Drop rows where linkedin_url is missing (if inner join behavior is desired)
atium_df = atium_df.dropna(subset=['linkedin_url'])

In [31]:
len(atium_df)

14773

In [32]:
# Save the enriched dataframe
csv_out_file_path = wip_dir / 'enriched_companies.csv'
atium_df.to_csv(csv_out_file_path, index=False)